# Binary Search  
**Def**: To find a solution through narrowing the search area by halves.  
  
## Sorted Array  

In a sorted array, the method we saw in the age guessing game works well. That is, start from the left most index and right most index, take the average to get the median index, if the target is greater than (or less than) said index, update the left most (right most) index accordingly. 

In [2]:
def bin_search(array, tgt):
    right = len(array) - 1
    left = 0
    while right >= left:
        mid = (right + left) // 2
        if array[mid] == tgt: return mid
        elif array[mid] > tgt: right = mid - 1
        elif array[mid] < tgt: left = mid + 1

    return "Not found"

# test
print(bin_search([1,2,3, 4, 5], 4 ))



3


The above code takes $k$ times to guess an array of length $2^k$. Then, since $N = 2^k$, the number of computations it undergoes is $k = \log_2 N \approx \log N$ making the complexity 
$$O(\log N)$$

A more generalized def of binary search.  
**Def**: For each integer $x$ we are give a boolean condition $P$. Of the integers, we are given integers $l$ and $r$ where $l < r$, and the below holds: 
* $P(l) =$ false  
* $P(r) =$ true
* $\exists M \; (l < M \le r) \ni$
    + $\forall x < M, P(x) = \text{false}$
    + $\forall x \ge M, P(x) = \text{true}.$  
    
In this case, we set $D= r - l$ and the binary search algorithm can identify $M$ in $O(\log D)$

In [ ]:
def gen_bin_search(array, P):
    """
    Inputs:
      array (lst): arrray to search
      P (function): boolean function that evaluates elements in array.
    Returns the lowest integer where P evaluates to true. 
    """
    left = array[0]
    right = array[-1]
    while (right - left > 1):
        mid = left + (right - left) / 2
        if P(mid): 
            right = mid
        else:
            left = mid
    return right

In [ ]:
def gen_bin_search(array, P, err):
    """
    Inputs:
      array (lst): arrray to search
      P (function): boolean function that evaluates elements in array.
      err (float): tolerated error
    Returns the lowest integer where P evaluates to true. 
    """
    left = array[0]
    right = array[-1]
    while (right - left > err):
        mid = left + (right - left) / 2
        if P(mid): 
            right = mid
        else:
            left = mid
    return right

A more generalized approach:  

**Def**: We are given a continuous function $f(x)$ over an interval in $\mathbb{R}$. Given two points in that interval $l, r$ where $l < r$, $f(l)$ and $f(r)$ have different signs (polarity). Then, using the binary method, we can find at least one $x$ such that $f(x) = 0$ at any level of precision. The existence of such a point is proven by the *intermediate value theorem*. 

## Minimum Value of Pair Sum with Lower Bound  

The problem can be stated as such: 
> Suppose we have N integers represented by $a_i\;(i= 0, ..., N-1)$, and another $N$ integers represented by $b_i \;(i=0, ..., N-1)$.  
> Suppose we choose an integer from each sequence and calculate its sum.  
> Of all possible sums, determine the minimum sum that is above a certain threshold $K$. 

To solve this, first we sort the elements in $b_i$. This takes $O(N\log N)$.   
Then, we fix $a_i$, and iterate through the lowest element in $b_i$ such that $b_i \ge K - a_i$.  

By using our abovemetioned general binary search algorithm, we can get the lower bound of $b_i$ in $O(\log N)$. Since we repeat this for each $a_i$, which there are $N$ of, the overall time complexity is $O(N \log N)$. Added to the earlier sorting time complexity, this remains the same (scalar multiples do not factor in). 

In [32]:
import math
import bisect

def pair_sum_min(a, b, K):
    min_sum = math.inf
    b = sorted(b)
    n = len(b)
    for ai in a:
        i = bisect.bisect_left(b, K - ai) # lowerbound function
        #print(ai, bi)
        if i < n:
            min_sum = min(min_sum, ai + b[i])
    return min_sum

print(pair_sum_min([3, 4, 5, 6, 5, 7], [10, 20, 30, 14, 50, 34], 10))


9
13
0


## From Optimization to Classification  

We convert the last optimization problem "find optimum of $x$ that satisfies such and such condition$ to a classification problem "determine whether $x$ satisifies condition$. Then, we can use the binary algorithm to locate the optimum in $O(\log N)$ time.  

### Shooting King  
>There are $N$ baloons each with initial altitude $H_i$, and rising $S_i$ per second. The objective is to shoot and pop every of these balloons.  
>You are allowed to pop one balloon the moment the game starts, and from there you may pop 1 balloon per second. The order of balloons you pop is up to you.   
>However, there will be a penalty for popping each balloon, which is its altitude, and the overall penalty is the maximum penalty you incur during the game.  
>You would like to minimize this penalty whilst popping all balloons.   
>>$H_i, S_i > 0$

The trick is to convert the above optimization to a classification problem:  
> Given $x$, is it possible to pop all balloons while keeping the penalty below $x$ for each one?  

Once we code the solution to the above, the actual solution can be derived via binary search as below:  
> Let left = 0, right = $\max (H_0 + (N-1)S_0, \cdots, H_{N-1} + (N-1) S_{N-1})$ (max possible altitude when the game ends at $t = N-1$).   
>The solution here is $O(\log(right - left))$




In [ ]:
def can_pop(x, h, s):
    """
    Sorting takes O(NlogN)
    Everything else is O(N)
    """
    res = True
    n = len(h)
    time_left = [0] * n
    # populate time left to pop each balloon
    for i in range(n):
        if h[i] > x: # the initial height already violates upper bound
            return False
        else:
            time_left[i] = (x - h[i]) / s[i] # dist left to hit upper bound divided by rate of ascension
    time_left = sorted(time_left)
    for t in range(n):
        # pop balloons based on their time left
        # one balloon each second 
        # if t > time_left, that means we did not pop in time
        if t > time_left[t]:
            res = False
            break

    return res

def shooter_min(h, s):
    n = len(h)
    left = 0
    right = max([h[i] + (n-1) * s[i] for i in range(n)])
    while (right - left) > 1:
        mid = (left + right) / 2
        if can_pop(mid, h, s):
            right = mid
        else:
            left = mid
    return right



In the above program, can_pop takes $O(N\log N)$ and the binary search takes $O(\log M)$ where $M = right - left$. So overall, 
$$O(N\log N \log M)$$

What we know from the above problem is that it is formatted in a way such that we are minimizing the maximum of $N$ variables (mini-max). This is a common optimization problem, applicable to problems such as minimizing the maximum number of working hours of $N$ workers. 